In [39]:
import folium
import pandas as pd
folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="CartoDB dark_matter")
marker = folium.CircleMarker(location=[40.738, -73.98])
marker.add_to(folium_map)

In [40]:
folium_map

Let's add some data! by reading in some excel data via pandas!

In [12]:
bike_data = pd.read_csv("201306-citibike-tripdata.csv")
bike_data["starttime"] = pd.to_datetime(bike_data["starttime"])
bike_data["stoptime"] = pd.to_datetime(bike_data["stoptime"])
bike_data["hour"] = bike_data["starttime"].map(lambda x: x.hour)

In [14]:
bike_data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,hour
0,695,2013-06-01 00:00:01,2013-06-01 00:11:36,444,Broadway & W 24 St,40.742354,-73.989151,434.0,9 Ave & W 18 St,40.743174,-74.003664,19678,Subscriber,1983.0,1,0
1,693,2013-06-01 00:00:08,2013-06-01 00:11:41,444,Broadway & W 24 St,40.742354,-73.989151,434.0,9 Ave & W 18 St,40.743174,-74.003664,16649,Subscriber,1984.0,1,0
2,2059,2013-06-01 00:00:44,2013-06-01 00:35:03,406,Hicks St & Montague St,40.695128,-73.995951,406.0,Hicks St & Montague St,40.695128,-73.995951,19599,Customer,NaN,0,0
3,123,2013-06-01 00:01:04,2013-06-01 00:03:07,475,E 15 St & Irving Pl,40.735243,-73.987586,262.0,Washington Park,40.691782,-73.973730,16352,Subscriber,1960.0,1,0
4,1521,2013-06-01 00:01:22,2013-06-01 00:26:43,2008,Little West St & 1 Pl,40.705693,-74.016777,310.0,State St & Smith St,40.689269,-73.989129,15567,Subscriber,1983.0,1,0


Net Arrivals/Departures
For this example we will try to explore if there is net migration of bikes from one bike station to another and if this migration depends on the time of day. To get started we will create a DataFrame containing the locations of each station.

In [17]:
# select the first occurrence of each station id
locations = bike_data.groupby("start station id").first()
# and select only the tree columns we are interested in
locations = locations.loc[:, ["start station latitude",
                              "start station longitude",
                              "start station name"]]

Select one hour of the day, and count trips with the same departure point.



In [27]:
selected_hour = 1
subset = bike_data[bike_data["hour"]==selected_hour]
departure_counts =  subset.groupby("start station id").count()
# select one column
departure_counts = departure_counts.iloc[:,[0]]
# and rename that column
departure_counts.columns= ["Departure Count"]

In [28]:
subset = bike_data[bike_data["hour"]==selected_hour]
arrival_counts =  subset.groupby("end station id").count()
# select one column
arrival_counts = arrival_counts.iloc[:,[0]]
# and rename that column
arrival_counts.columns= ["Arrival Count"]

Next we join the arrival counts, departure counts and locations into one table.

In [30]:
trip_counts = departure_counts.join(locations).join(arrival_counts)


In [32]:
for index, row in trip_counts.iterrows():
    net_departures = (row["Departure Count"]-row["Arrival Count"])
    radius = net_departures/20
    if net_departures>0:
        color="#E37222" # tangerine
    else:
        color="#0A8A9F" # teal
    
    folium.CircleMarker(location=(row["start station latitude"],
                                  row["start station longitude"]),
                        radius=radius,
                        color=color,
                        fill=True).add_to(folium_map)

In [33]:
folium_map # when selected_hour was 1

In [34]:
selected_hour = 8
subset = bike_data[bike_data["hour"]==selected_hour]
departure_counts =  subset.groupby("start station id").count()
# select one column
departure_counts = departure_counts.iloc[:,[0]]
# and rename that column
departure_counts.columns= ["Departure Count"]
subset = bike_data[bike_data["hour"]==selected_hour]
arrival_counts =  subset.groupby("end station id").count()
# select one column
arrival_counts = arrival_counts.iloc[:,[0]]
# and rename that column
arrival_counts.columns= ["Arrival Count"]
trip_counts = departure_counts.join(locations).join(arrival_counts)
for index, row in trip_counts.iterrows():
    net_departures = (row["Departure Count"]-row["Arrival Count"])
    radius = net_departures/20
    if net_departures>0:
        color="#E37222" # tangerine
    else:
        color="#0A8A9F" # teal
    
    folium.CircleMarker(location=(row["start station latitude"],
                                  row["start station longitude"]),
                        radius=radius,
                        color=color,
                        fill=True).add_to(folium_map)


In [35]:
folium_map # when selected_hour was 8